In [ ]:
# Container Debugging Toolkit

Use the cells below to check logs and manually start services if they didn't come up automatically.

**How to Use:**
1. Select a cell.
2. Click the 'Run' button (▶) in the toolbar above, or press Shift+Enter.

In [ ]:
## 1. Check Supervisor Status

Supervisor is the main process manager. Let's see if it's running and what services it *thinks* are running.

In [ ]:
%%bash
# Check if supervisord process is running
echo "--- Checking for supervisord process ---"
ps aux | grep supervisord | grep -v grep || echo "Supervisord process NOT found!"

# Use supervisorctl (the command-line client) to get status
# This requires supervisord to be running and configured correctly
echo "\n--- Supervisorctl Status ---"
supervisorctl status || echo "Error running supervisorctl. Is supervisord running?"

In [ ]:
## 2. Check Main Container Logs (Supervisor Output)

Supervisor redirects the output (stdout/stderr) of the services it manages. This command shows the *end* of the main supervisor log, which often contains startup errors for the services.

In [ ]:
%%bash
# Supervisor logs to stdout/stderr by default in our config, which Docker captures.
# This cell can't directly access those Docker logs easily.
# INSTEAD: Check the main container logs via the Vast.ai Web UI or `docker logs <container_id>` if running locally.
echo "Please check the main container logs via the Vast.ai Web UI."
echo "Supervisor is configured to send logs there."
echo "Look for output AFTER the 'Provisioning Script Finished' message."

# If supervisor config was changed to log to files (e.g., /logs):
# echo "\n--- Last 50 lines of /logs/supervisord.log ---"
tail -n 50 /logs/supervisord.log || echo "/logs/supervisord.log not found."

In [ ]:
## 3. Check On-Start Provisioning Script Log

This shows the log file created by the `provisioning_script.sh` that ran when the container first started. Useful for checking if cloning, TensorRT install, or model downloads had errors.

In [8]:
%%bash
# Adjust path if using /workspace or /app instead of /
LOG_PATH="/logs/onstart_script.log" # Or /workspace/onstart_script.log or /logs/onstart_script.log
echo "--- Contents of $LOG_PATH ---"
cat "$LOG_PATH" || echo "$LOG_PATH not found."

5M 0s
249500K .......... .......... .......... .......... .......... 92% 1.67M 0s
249550K .......... .......... .......... .......... .......... 92%  201M 0s
249600K .......... .......... .......... .......... .......... 92%  103M 0s
249650K .......... .......... .......... .......... .......... 92%  168M 0s
249700K .......... .......... .......... .......... .......... 92%  171M 0s
249750K .......... .......... .......... .......... .......... 92%  142M 0s
249800K .......... .......... .......... .......... .......... 92%  168M 0s
249850K .......... .......... .......... .......... .......... 92%  173M 0s
249900K .......... .......... .......... .......... .......... 92%  159M 0s
249950K .......... .......... .......... .......... .......... 92%  174M 0s
250000K .......... .......... .......... .......... .......... 92%  134M 0s
250050K .......... .......... .......... .......... .......... 92%  175M 0s
250100K .......... .......... .......... .......... .......... 92%  136M 0s
250150

In [ ]:
## 4. Check VNC Window Manager Log

The `wm_startup.sh` script (which starts IceWM) creates its own log file. Check this if the VNC connects but you only see a gray screen.

In [3]:
%%bash
# Path depends on where wm_startup.sh was copied and if HOME is /root or /workspace etc.
# Assuming it was copied to /etc/icewm/wm_startup.sh and HOME is /root
LOG_PATH="/root/wm.log" # Default VNC log path if started by user
echo "--- Contents of $LOG_PATH ---"
cat "$LOG_PATH" || echo "$LOG_PATH not found. Did the WM startup script run?"

# Check supervisor VNC logs if configured for file logging:
# echo "\n--- Last 50 lines of /logs/vnc.log ---"
# tail -n 50 /logs/vnc.log || echo "/logs/vnc.log not found."
# echo "\n--- Last 50 lines of /logs/vnc_err.log ---"
tail -n 50 /logs/vnc_err.log || echo "/logs/vnc_err.log not found."

--- Contents of /root/wm.log ---


cat: /root/wm.log: No such file or directory


/root/wm.log not found. Did the WM startup script run?


tail: cannot open '/logs/vnc_err.log' for reading: No such file or directory


/logs/vnc_err.log not found.


In [ ]:
## 5. Manually Start VNC Service

If VNC isn't running (check `supervisorctl status` first!), you can try starting it manually using the same script Supervisor uses. Run this cell, then try connecting with your VNC client after a few seconds.

In [1]:
%%bash
echo "--- Attempting to manually start VNC using the main startup script ---"
# Run the script in the background (&) so it doesn't block the notebook
# Make sure the path is correct
bash /dockerstartup/vnc_startup.sh --wait &
echo "Command sent to background. Check VNC connection after a few seconds."
# You might need to check logs again if it fails.

--- Attempting to manually start VNC using the main startup script ---
Command sent to background. Check VNC connection after a few seconds.

------------------ change VNC password  ------------------

---------  purging existing VNC password settings  ---------

------------------ start noVNC  ----------------------------
start vncserver with param: VNC_COL_DEPTH=24, VNC_RESOLUTION=1024x768
...
start window manager
...


In [ ]:
## 6. Locate Key Scripts

Just lists the locations of the main scripts involved based on the final Dockerfile setup.

In [ ]:
%%bash
echo "--- Key Script Locations ---"
echo "On-Start Provisioning Script (copied into image): /provisioning_script.sh" # Or /workspace/... or /app/...
echo "Main VNC Startup Script (run by Supervisor): /dockerstartup/vnc_startup.sh"
echo "IceWM Startup Script (run by VNC script): /etc/icewm/wm_startup.sh" # Or /workspace/wm_startup.sh etc.
echo "Supervisor Config: /etc/supervisor/conf.d/supervisord.conf"
echo "Log directory (if file logging enabled): /logs/" # Or /workspace/logs or /app/logs